<a href="https://colab.research.google.com/github/wdavid93/OpenClassRoom/blob/main/test_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!ls

sample_data  spark-3.1.1-bin-hadoop3.2	spark-3.1.1-bin-hadoop3.2.tgz


In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [4]:
!wget https://s3.eu-west-1.amazonaws.com/course.oc-static.com/projects/Data_Scientist_P8/fruits.zip

--2023-12-06 10:11:30--  https://s3.eu-west-1.amazonaws.com/course.oc-static.com/projects/Data_Scientist_P8/fruits.zip
Resolving s3.eu-west-1.amazonaws.com (s3.eu-west-1.amazonaws.com)... 52.218.40.19, 52.218.24.243, 52.218.92.123, ...
Connecting to s3.eu-west-1.amazonaws.com (s3.eu-west-1.amazonaws.com)|52.218.40.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1379678841 (1.3G) [application/zip]
Saving to: ‘fruits.zip’

fruits.zip          100%[===================>]   1.28G  35.9MB/s    in 38s     

2023-12-06 10:12:08 (34.4 MB/s) - ‘fruits.zip’ saved [1379678841/1379678841]



In [5]:
!unzip -q fruits.zip

In [8]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.ml.image import ImageSchema

spark = SparkSession.builder.master("local[*]").getOrCreate()
# Charger les images
images_df = spark.read.format("image").load("fruits-360_dataset/fruits-360/Test/Apple Braeburn")

In [9]:
images_df.printSchema()

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)



In [10]:
images_df.count()

164

In [19]:
from pyspark.sql import SparkSession
import os

spark = SparkSession.builder.master("local[*]").getOrCreate()

# Chemin d'accès au répertoire des images
image_dir = "fruits-360_dataset/fruits-360/Test/Apple Braeburn"

# Vérifier si le répertoire existe
if os.path.exists(image_dir):
    # Charger les images
    images_df = spark.read.format("image").load(image_dir)
    # Afficher les premières lignes de images_df
    # images_df.show()
    print(f"Le répertoire {image_dir} existe.")
    # images_df.show()
else:
    print(f"Le répertoire {image_dir} n'existe pas.")


Le répertoire fruits-360_dataset/fruits-360/Test/Apple Braeburn existe.


In [21]:
!ls -l 'fruits-360_dataset/fruits-360/Test/Apple Braeburn/r_4_100.jpg'


-rw-r--r-- 1 root root 5473 Sep 12  2021 'fruits-360_dataset/fruits-360/Test/Apple Braeburn/r_4_100.jpg'


In [24]:
#import pyspark
import time
from pyspark import SQLContext
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.image import ImageSchema

from pyspark.sql.functions import udf
from pyspark.sql.functions import input_file_name
from pyspark.sql.types import *
#import pyspark

def parse_categorie(path):
    '''Renvoie la catégorie d\'une image à partir de son chemin'''
    if len(path) > 0:
        #catégorie de l'image
        return path.split('/')[-2]
    else:
        return ''

def import_dossier(path):
    '''Renvoie un dataframe spark des images à partir du chemin du dossier.
    :param path: chemin vers le dossier
    :return: dataframe spark contenant toutes les images du dossier'''


def load_data(path_img):
    '''Chargement des dataframes:
    Prend en entrée le répertoire qui contient les sous répertoires contenant les images
    Renvoie en sortie un spark dataframe contenant les images et
    un spark dataframe contenant les noms des fruits associés'''
    #compteur
    start = time.time()
    #chargement dataframe des images

    df_img = spark.read.format("image").load(path) # ne fonctionne pas si il y a des espaces dans le chemin
    #df_img =  ImageSchema.readImages(path_img, dropImageFailures = True)
    print('chargement effectué')
    #récupération chemin à partir des images
    df_img = df_img.withColumn("path", input_file_name())
    #catégorisation des images
    udf_categorie = udf(parse_categorie, StringType())
    df_img = df_img.withColumn('categorie', udf_categorie('path'))
    print('Temps de chargement des images : {} secondes'.format(time.strftime('%S', time.gmtime(time.time()-start))))

    return df_img

In [25]:
#chargement du dataframe contenant les images, leur chemin et leur catégorie
path = "fruits-360_dataset/fruits-360/Test/Apple Braeburn"
spark_df = load_data(path)

chargement effectué
Temps de chargement des images : 00 secondes


In [26]:
spark_df.count()

164

In [27]:
import tensorflow as tf
from pyspark.sql.functions import col, udf
from pyspark.sql.types import ArrayType, FloatType
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import Model

# Charger le modèle ResNet50 pré-entraîné sans la dernière couche de classification
# base_model = ResNet50(weights='imagenet', include_top=False)
# model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)
# Chargement du modèle ResNet50 sans poids
model = ResNet50(weights=None)

# Fonction pour charger et prétraiter l'image
def preprocess_image(image_path):
    # Charger l'image
    img = load_img(image_path, target_size=(224, 224))
    # Convertir en tableau numpy
    img_array = img_to_array(img)
    # Ajouter une dimension pour créer un batch de taille 1
    img_batch = np.expand_dims(img_array, axis=0)
    # Prétraiter l'image pour le modèle ResNet50
    img_preprocessed = preprocess_input(img_batch)
    return img_preprocessed

# Fonction UDF pour appliquer le modèle à l'image et obtenir les caractéristiques
def extract_features(image_path):
    img_preprocessed = preprocess_image(image_path)
    features = model.predict(img_preprocessed)
    # Aplatir les caractéristiques en une liste pour les stocker dans une colonne DataFrame
    return features.flatten().tolist()

# Enregistrer la fonction UDF avec le type de retour approprié
extract_features_udf = udf(extract_features, ArrayType(FloatType()))

def preprocess_data(dataframe):
    '''Renvoie le résultat de l'avant dernière couche de chaque image du dataframe via le modèle ResNet50
    return un df contenant des vecteurs de dimension 1x2048 '''

    # Appliquer la fonction UDF pour extraire les caractéristiques
    output = dataframe.withColumn('image_preprocessed', extract_features_udf(col('path')))
    # Sélectionner les colonnes nécessaires
    output = output.select(['path', 'categorie', 'image_preprocessed'])
    return output


In [28]:
spark_df_preprocessed = preprocess_data(spark_df)

In [30]:
spark_df_preprocessed.count()

164